In [ ]:
from spotmax import pipe

In [ ]:
result = pipe.spots_semantic_segmentation(
    image, 
    lab=None,
    gauss_sigma=0.0,
    spots_zyx_radii_pxl=None, 
    do_sharpen=False, 
    do_remove_hot_pixels=False,
    lineage_table=None,
    do_aggregate=True,
    use_gpu=False,
    logger_func=print,
    thresholding_method=None,
    keep_input_shape=True,
    nnet_model=None,
    nnet_params=None,
    nnet_input_data=None,
    bioimageio_model=None,
    bioimageio_params=None,
    do_preprocess=True,
    do_try_all_thresholds=True,
    return_only_segm=False,
    pre_aggregated=False,
    raw_image=None
)